In [1]:
import numpy as np
from steps import *
from tqdm import tqdm
from optimal import *
import matplotlib.pyplot as plt
import optuna
import pickle
import warnings
warnings.filterwarnings("error")


In [2]:
def plot_res(x_axis, results, x_label, y_label, label, title, path):
    plt.grid()
    plt.plot(x_axis, results.mean(axis=1), label=label)
    plt.fill_between(x_axis, results.mean(axis=1)+results.std(axis=1), results.mean(axis=1)-results.std(axis=1),
                     color='r', alpha=0.5)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig(path+".png")
    plt.clf()

    plt.grid()
    plt.boxplot(results.T, positions=x_axis)
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    plt.legend()
    plt.savefig("BP-"+path+".png")
    plt.clf()

In [3]:
d = 20
opt_r = 5
m = 20
T_r = 100
times = 10
opt_sigma = 1
sigma_type = matrix_type.DIAG
s_type = matrix_type.IDENTITY

name = f"optuna_optimization_d={d}_r={opt_r}_sigma={sigma_type}_s={s_type}_m={m}_Tr={T_r}_var={opt_sigma}"
study = optuna.create_study(study_name=f'{name}',
                            storage=f'sqlite:///./../optuna/{name}.db ',
                            direction='minimize', load_if_exists=True)

[I 2023-05-12 20:11:39,136] Using an existing study with name 'optuna_optimization_d=20_r=5_sigma=matrix_type.DIAG_s=matrix_type.IDENTITY_m=20_Tr=100_var=1' instead of creating a new one.


In [ ]:
# Sigma
sigma_max = 10.0
sigma_min = 0.5
spaces = 0.5
load = False

suffix = f'{sigma_min}-{sigma_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    d_sigmas = np.linspace(sigma_min, sigma_max, int((sigma_max-sigma_min)/spaces)+1)
    results = np.zeros([len(d_sigmas), times])
    for idx, d_sigma in enumerate(d_sigmas):
        print(f" ***** Sigma={d_sigma} *****")
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=d_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_sigma0_{suffix}.p', 'wb'))
    pickle.dump(d_sigmas, open(f'sigma0-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_sigma0_{suffix}.p', 'rb'))
    d_sigmas = pickle.load(open(f'sigma0-acc_{suffix}.p', 'rb'))

plot_res(x_axis=d_sigmas, results=results,
         x_label=r'$\sigma_0$', y_label='Accuracy Ratio', label=f'd={d}, r={opt_r}',
         title=r'results accuracy ratio vs $\sigma_0$',
         path=f"results_accuracy_ratio_sigma0_{suffix}")

 ***** Sigma=0.5 *****
l_star=15 ; regret_mix=0.7304976777765652 ; last_regret=0.8643475856377519
l_star=14 ; regret_mix=0.7035713161429785 ; last_regret=0.8116785243431476
l_star=15 ; regret_mix=0.6933758819668112 ; last_regret=0.816793297072002
l_star=12 ; regret_mix=0.6888220298237453 ; last_regret=0.8236106617471113
l_star=18 ; regret_mix=0.6742239569665369 ; last_regret=0.7711118555797836
l_star=14 ; regret_mix=0.709879395657479 ; last_regret=0.8471496950399728
l_star=13 ; regret_mix=0.6877988959735862 ; last_regret=0.8232323293091035
l_star=15 ; regret_mix=0.6774051896194873 ; last_regret=0.800398843127305
l_star=12 ; regret_mix=0.6971422803241883 ; last_regret=0.8381399944410003
l_star=14 ; regret_mix=0.6881012829791996 ; last_regret=0.8088472849504943
 ***** Sigma=1.0 *****
l_star=12 ; regret_mix=0.5648746863196206 ; last_regret=0.6546822312270139
l_star=9 ; regret_mix=0.3376828865304244 ; last_regret=0.4160944934706894
l_star=11 ; regret_mix=0.6079226857694537 ; last_regret=0.

In [ ]:
# r
r_max = 10
r_min = 1
spaces = 1
load = False

suffix = f'{r_min}-{r_max}-{spaces}_opt-r={opt_r}_opt-sigma={opt_sigma}'

if not load:
    r_list = range(r_min, r_max, spaces)
    results = np.zeros([len(r_list), times])
    for idx, r in enumerate(r_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_r_.p', 'wb'))
    pickle.dump(r_list, open(f'r-acc_{suffix}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_r_{suffix}.p', 'rb'))
    r_list = pickle.load(open(f'r-acc_{suffix}.p', 'rb'))

plot_res(x_axis=r_list, results=results,
         x_label='r', y_label='Accuracy Ratio', label=f"d={d}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs r',
         path=f"results_accuracy_ratio_r_{suffix}")


In [ ]:
# d
load = False
d_max = 20
d_min = opt_r+2
spaces = 1

if not load:
    d_list = range(d_min, d_max, spaces)
    results = np.zeros([len(d_list), times])

    for idx, d in enumerate(d_list):
        regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
        ratio = regret_list.min(axis=1) / regret_mix_list
        results[idx, :] = ratio

    pickle.dump(results, open(f'acc-ratio_vs_d_{d_min}-{d_max}-{spaces}.p', 'wb'))
    pickle.dump(r_list, open(f'd-acc_{d_min}-{d_max}-{spaces}.p', 'wb'))

else:
    results = pickle.load(open(f'acc-ratio_vs_d_{d_min}-{d_max}-{spaces}.p', 'rb'))
    d_list = pickle.load(open(f'd-acc_{d_min}-{d_max}-{spaces}.p', 'rb'))

plot_res(x_axis=d_list, results=results,
         x_label='d', y_label='Accuracy Ratio', label=f"r={opt_r}"+r" $\sigma_0$="+f"{opt_sigma}",
         title='results accuracy ratio vs d',
         path=f"results_accuracy_ratio_d_{d_min}-{d_max}-{spaces}")

In [ ]:
# Regrets
load = False

if not load:
    d=20
    regret_list, regret_mix_list = run_algorithm(d=d, r=opt_r, m=m, sigma_type=sigma_type, s_type=s_type,
                                                     times=times, d_sigma=opt_sigma, T_r=T_r,
                                                     **study.best_params)
    pickle.dump(regret_list, open(f'regrets.p', 'wb'))

else:
    regret_list = pickle.load(open(f'regrets.p', 'rb'))


plot_res(x_axis=range(m+2), results=regret_list.T,
         x_label='#Iteration', y_label='Regret', label=f'd={d} r={opt_r} '+r'$\sigma_0$='+f'{opt_sigma}',
         title='Regret vs #Iteration',
         path="Regret_vs_Iteration")
